In [1]:
%pip install duckduckgo-search
%pip install wikipedia
%pip install langchain
%pip install langchainhub
%pip install langchain_openai

Note: you may need to restart the kernel to use updated packages.

Note: you may need to restart the kernel to use updated packages.
  Using cached urllib3-2.2.2-py3-none-any.whl.metadata (6.4 kB)
Using cached urllib3-2.2.2-py3-none-any.whl (121 kB)
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.26.19
    Uninstalling urllib3-1.26.19:
      Successfully uninstalled urllib3-1.26.19
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
alpaca-trade-api 3.2.0 requires urllib3<2,>1.24, but you have urllib3 2.2.2 which is incompatible.
flair 0.13.1 requires urllib3<2.0.0,>=1.0.0, but you have urllib3 2.2.2 which is incompatible.
transformer-smaller-training-vocab 0.3.3 requires sentencepiece<0.2.0,>=0.1.97, but you have sentencepiece 0.2.0 which is incompatible.
zetascale 2.4.5 requires tqdm==4.66.2, but you have tqdm 4.66.4 which is incompatible.
zetascale 2.4.5 requires transformers==4.39.1, but you have transformers 4.43.4 which is incompatible.


Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: langchain_openai in c:\users\mudit\appdata\local\programs\python\python311\lib\site-packages (0.1.20)
  Attempting uninstall: openai
    Found existing installation: openai 0.28.0
    Uninstalling openai-0.28.0:
      Successfully uninstalled openai-0.28.0



ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
csv-agent 0.0.1 requires langchain<0.2,>=0.1, but you have langchain 0.2.14 which is incompatible.
csv-agent 0.0.1 requires pydantic<2, but you have pydantic 2.8.2 which is incompatible.
csv-agent 0.0.1 requires tiktoken<0.6.0,>=0.5.1, but you have tiktoken 0.7.0 which is incompatible.
llama-index-vector-stores-chroma 0.1.6 requires chromadb<0.5.0,>=0.4.22, but you have chromadb 0.3.29 which is incompatible.


In [12]:
import os
os.environ["GOOGLE_API_KEY"]=""
os.environ["LANGCHAIN_API_KEY"]=""

In [4]:
from langchain import hub
from langchain.agents import create_openai_functions_agent
from langchain.agents import AgentExecutor
from langchain.schema import HumanMessage, SystemMessage
from langchain_google_genai import ChatGoogleGenerativeAI


In [52]:
llm=ChatGoogleGenerativeAI(model="gemini-1.5-pro", temperature=0)

In [9]:
messages = [
    SystemMessage(
        content="A User will input in a year and you will get the baseball world series champion"
    ),
    HumanMessage(
        content="2023"
    ),
]
llm.invoke(messages)

AIMessage(content='The 2023 World Series Champions were the **Houston Astros**! \n', response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-975b5b2a-7f04-4f0a-9128-a2f922d4c58d-0', usage_metadata={'input_tokens': 21, 'output_tokens': 16, 'total_tokens': 37})

In [10]:
from langchain.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [13]:
prompt = hub.pull("hwchase17/openai-functions-agent")

In [14]:
api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=500)
tools = [WikipediaQueryRun(api_wrapper=api_wrapper)]

In [15]:
agent = create_openai_functions_agent(llm, tools, prompt)

In [16]:
agent_executor = AgentExecutor(agent=agent, tools=tools)

In [18]:
agent_executor.invoke({"input": "Who won the 2023 world series?"})


{'input': 'Who won the 2023 world series?',
 'output': 'The Houston Astros won the 2023 World Series, defeating the Philadelphia Phillies in five games. \n'}

In [19]:
from langchain import LLMMathChain
from langchain.agents import Tool, initialize_agent
from langchain.tools import DuckDuckGoSearchRun

In [45]:
from langchain.agents import Tool
from langchain.tools import tool


In [46]:
@tool
def calculator(expression: str) -> str:
    """Useful for when you need to answer questions about math. This tool is only for math questions and nothing else. Input should be a math expression."""
    llm_math_chain = LLMMathChain.from_llm(llm=llm, verbose=True)
    return llm_math_chain.run(expression)

In [35]:
search = DuckDuckGoSearchRun()


In [47]:
tools2 = [calculator, search]


In [48]:
from langchain.agents import AgentType, initialize_agent

In [49]:
agent2 = initialize_agent(
    tools, 
    llm, 
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)


In [55]:
agent2.invoke("Who won the 2021 world series and how many years was it since their last world series win")



> Entering new AgentExecutor chain...
Thought: I need to use wikipedia to find out who won the world series in 2021 and when they last won.
Action: wikipedia
Action Input: 2021 World Series
Observation: Page: 2021 World Series
Summary: The 2021 World Series was the championship series of Major League Baseball's (MLB) 2021 season. The 117th World Series was a best-of-seven playoff between the National League (NL) champion Atlanta Braves and the American League (AL) champion Houston Astros. The series began on October 26 and concluded on November 2.
The Braves won the series four games to two. It was their fourth World Series title in franchise history, their first since 1995, and their second si
Thought:Thought: I need to find out when the Atlanta Braves last won the World Series before 2021. 
Action: wikipedia
Action Input: Atlanta Braves
Observation: Page: Atlanta Braves
Summary: The Atlanta Braves are an American professional baseball team based in the Atlanta metropolitan area. Th

Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised InternalServerError: 500 An internal error has occurred. Please retry or report in https://developers.generativeai.google/guide/troubleshooting.


Thought: The Atlanta Braves won the World Series in 1995. 2021-1995=26 years.
Final Answer: The Atlanta Braves won the 2021 World Series. It was 26 years since their last win in 1995. 


> Finished chain.


{'input': 'Who won the 2021 world series and how many years was it since their last world series win',
 'output': 'The Atlanta Braves won the 2021 World Series. It was 26 years since their last win in 1995.'}